In [2]:
%classpath config resolver scijava.public https://maven.scijava.org/content/groups/public
%%classpath add mvn
net.imagej imagej 2.0.0-rc-71
net.imagej imagej-notebook 0.7.1

SyntaxError: invalid syntax (<ipython-input-2-129086478889>, line 3)

In [ ]:
ij = new net.imagej.ImageJ()
"ImageJ v${ij.getVersion()} is ready to go."

In [ ]:
#Load an image
image = ij.io().open("https://imagej.net/images/lymp.tif")

In [ ]:
#Compute Histogram
histogram = ij.op().image().histogram(image)

In [ ]:
#N-dimnesional crop
mandrill = ij.io().open("https://imagej.net/images/baboon.jpg")

In [ ]:
import net.imglib2.FinalInterval
min = [85, 0, 110]
len = [5, 335, 1]
bounds = FinalInterval.createMinSize(min[0], len[0], min[1], len[1], min[2], len[2])
eyes = ij.op().transform().crop(mandrill, bounds, true)

In [ ]:
#Type conversion
eyes32 = ij.op().convert().float32(eyes)

In [ ]:
eyes.firstElement().getClass().getName()

In [ ]:
eyes32.firstElement().getClass().getName()

In [ ]:
#Median filter with circular neighborhood
import net.imglib2.algorithm.neighborhood.HyperSphereShape
median = ij.op().run("create.img", eyes32)
neighborhood = new HyperSphereShape(4)
ij.op().run("filter.median", median, eyes32, neighborhood)

In [ ]:
#Difference of Gaussians formulas

dogFormula = "gauss(image, sigma1) - gauss(image, sigma2)"
dog = ij.op().eval(dogFormula, [
  "image": eyes32,
  "sigma1": [20, 20],
  "sigma2": [4, 4]
])

In [ ]:
#Grayscale morphology operators

topHat = ij.op().morphology().topHat(eyes, [neighborhood])

In [ ]:
blackTopHat = ij.op().morphology().blackTopHat(eyes, [neighborhood])

In [ ]:
#Display multiple image at once
ij.notebook().display(["median":median, "topHat":topHat, "blackTopHat":blackTopHat])

In [ ]:
ij.notebook().display([["median":median, "topHat":topHat, "blackTopHat":blackTopHat]])

In [ ]:
#Fourier transform with lowpass filter

import net.imglib2.util.Util

lowpass = { fft, radius ->
  // Declare an array to hold the current position of the cursor.
  pos = new long[fft.numDimensions()]

  // Define origin as 0,0.
  long[] origin = [0, 0]

  // Define a 2nd 'origin' at bottom left of image.
  // This is a bit of a hack. We want to draw a circle around the origin,
  // since the origin is at 0,0 - the circle will 'reflect' to the bottom.
  long[] origin2 = [0, fft.dimension(1)]

  // Loop through all pixels.
  cursor = fft.localizingCursor()
  while (cursor.hasNext()) {
    cursor.fwd()
    cursor.localize(pos)

    // Calculate distance from 0,0 and bottom left corner
    // (so we can form the reflected semi-circle).
    dist = Util.distance(origin, pos)
    dist2 = Util.distance(origin2, pos)

    // If distance is above radius (cutoff frequency) set value of FFT to zero.
    if (dist > radius && dist2 > radius)
      cursor.get().setZero()
  }
}

In [ ]:

#perform a fast fourier transform (FFT) on the image, run the lowpass filter, then inver the FFT:

import net.imglib2.type.numeric.real.FloatType

// Perform fft of the input.
fft = ij.op().filter().fft(image)

// Filter it.
lowpass(fft, radius=10)

// Reverse the FFT.
inverse = ij.op().run("create.img", image, new FloatType())
ij.op().filter().ifft(inverse, fft)

// Display the result.
ij.notebook().display([["image":image, "lowpass":inverse]])